In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
data=pd.read_csv("../input/breast-cancer-wisconsin-data/data.csv")

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
del data['Unnamed: 32']

In [ ]:
data.tail()

In [ ]:
data.shape

In [ ]:
X=data.iloc[:,2:].values
Y=data.iloc[:,1].values

In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
encoder=LabelEncoder()
Y=encoder.fit_transform(Y)

Fitting into train test data

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=360)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

**Building an ANN**
* Importing keras libraries and packages

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
## Initialise the ANN
classifier=Sequential()


In [ ]:
## Build input and hidden layers

classifier.add(Dense(units=16,activation='relu',kernel_initializer='uniform',input_dim=30,name='1stlayer'))
classifier.add(Dropout(rate=0.1))

In [ ]:
classifier.add(Dense(units=16,activation='relu',kernel_initializer='uniform',name='2ndlayer'))
classifier.add(Dropout(rate=0.1))

In [ ]:
classifier.add(Dense(units=1,activation='sigmoid',kernel_initializer='uniform',name='outputlayer'))

In [ ]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
## fitting ANN to training dataset
history=classifier.fit(x_train,y_train,validation_split=0.33,batch_size=10,epochs=100)

In [ ]:
## predicting on test 
y_pred=classifier.predict(x_test)> 0.5

In [ ]:
## Evaluating using confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

In [ ]:
print(history.history.keys())

In [ ]:
# Visualizing the loss updates with the number of epochs
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training Loss vs Epochs')
plt.ylabel('Training Loss')
plt.xlabel('Number of Epochs')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# Visualizing the accuracy updates with the number of epochs
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training Accuracy vs Epochs')
plt.ylabel('Training Accuracy')
plt.xlabel('Number of Epochs')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 16,kernel_initializer = 'uniform', activation = 'relu', input_dim = 30))
    classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 15)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
accuracies = cross_val_score(estimator = classifier, X = x_train, y = y_train, cv = kfold)
print(accuracies)
mean = accuracies.mean()
print(mean)
variance = accuracies.std()
print(variance)

In [ ]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 30))
    classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [10, 25, 32],
              'epochs': [15, 20],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(x_train, y_train)    
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
print(best_parameters)

In [ ]:
# {'batch_size': 10, 'epochs': 100, 'optimizer': 'rmsprop'}
print(best_accuracy)